In [1]:
# $ pip install "tensorflow>=1.7.0"
# $ pip install tensorflow-hub

import numpy as np
import pandas as pd
import re
import tensorflow as tf
import tensorflow_hub as tfhub

wine_df = pd.read_csv("winemag-data-130k-v2.csv")

In [19]:
wine_df.head()
#a = wine_df.groupby(['country']).count() > 400
wine_df = wine_df.dropna(subset=['price'])
pd.set_option('display.max_colwidth', -1)

In [3]:
wine_df['price']= wine_df['price'].astype(str)
wine_df

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
6,6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,NaN,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
7,7,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
8,8,Germany,Savory dried thyme notes accent sunnier flavor...,Shine,87,12.0,Rheinhessen,NaN,NaN,Anna Lee C. Iijima,NaN,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel
9,9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam
10,10,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature


In [4]:
embed = tfhub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")

In [5]:
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  wine_embeddings = session.run(embed(list(wine_df.description)))
print(wine_embeddings.shape)

(120975, 512)


In [16]:
def recommend_engine(query, embedding_table = wine_embeddings):
    # Embed user query
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        embedding = session.run(embed([query]))

    # Calculate similarity with all reviews
    similarity_score = np.dot(embedding, embedding_table.T)
    
    # Recommend
    recommendations = wine_df.copy()
    recommendations['recommendation'] = similarity_metrics.T
    recommendations = recommendations.sort_values('recommendation', ascending=False)
    recommendations = recommendations[['variety', 'title', 'price', 'description', 'recommendation']]

    return recommendations

In [20]:
query = "dark, dry, robust"

recommendation1 = user_query_recommend(user_query1)
print(user_query1)

recommendation1.head(5).T

dark, dry, robust
Price range - [20, 60]


,87676,8421,112160,112892,94608
variety,Riesling,Cabernet Sauvignon,Grenache Blanc,Merlot-Cabernet Franc,Pinot Noir
title,Von Der Leyen 2013 Riesling (Nahe),Wellington 2010 Mohrhardt Ridge Vineyard Cabernet Sauvignon (Sonoma County),C R Graybehl 2014 Mounts Family Vineyards Grenache Blanc (Dry Creek Valley),Inspire Moore 2010 Truth Reserve Merlot-Cabernet Franc (Finger Lakes),Chime 2011 Pinot Noir (California)
price,11.0,30.0,24.0,45.0,12.0
description,"A touch of petrol lends a slick, mineral sheen to this lusciously peachy, yet zesty dry Riesling. Delicately textured yet mouth-filling, it finishes long on a lingering lime-zest note. Enjoy now for its youthful, primary nature","An earthy rusticity marks this wine, soft in black cherry, with a rich, smooth texture and grippy, tongue-coating finish of leathery tannin.","This is a soft, light-bodied white that refreshes in honeyed apple and apricot that's dry, pretty and quite floral, with a flat, linear finish.","Juicy black cherry and violet notes permeate throughout this easy-drinking Merlot-based blend. Luscious and fruity, but bold in style, it holds up well to the coating of grippy tannins.","This is minty and light, with a soft texture and red cherry tones. The finish turns overtly earthy, even barnyard-like."
recommendation,0.612814,0.608478,0.605926,0.603784,0.597814
